# import

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install lightly
! pip install wandb

In [3]:
import shutil
import os
def copyfile(file_list, folder=''):
  for file in file_list:
    shutil.copyfile(file, os.path.join(folder,os.path.basename(file)))

list_py = ['/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/model.py',
      '/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/utils.py',
      '/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/dataset.zip',
      '/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/masked_autoencoder.py']

copyfile(list_py)

In [4]:
from utils import extract_and_split

data_folder = 'dataset/dataset'
test_folder = 'test_set/test_set'
extract_and_split(zip_file_path = '/content/dataset.zip',
         extract_folder = 'dataset',
         test_folder = test_folder,
         data_folder = data_folder)


In [5]:
import torch
import torchvision
from torch import nn

from lightly.models import utils
import masked_autoencoder
from lightly.transforms.mae_transform import MAETransform
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

from   import MAE
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from IPython.display import Video
from tqdm.auto import tqdm, trange
from utils import MPJPELoss

# model

In [6]:
vit = torchvision.models.vit_b_16(weights=None)
model = MAE(vit, 64, 80)

device = "cuda"
model.to(device)

learning_rate = 1e-6
batch_size = 128
Epochs = 10

criterion = MPJPELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# wandb set

In [7]:
import wandb
from utils import normalize_data, Node_Dataset, load_mean_std, get_time
CHECKPOINT_PATH = '/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/state_dict/checkpoint.tar'
model_savepath = f'/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/state_dict'

mean, std = load_mean_std(data_folder)

wandb.init(
    project='MaE bs PJ',
    name=f'batch_size: {batch_size}',
    config={
        "learning_rate": learning_rate,
        "architecture": "ViT",
        "dataset": "customer",
        "epochs": Epochs,
        "mean": mean,
        "std": std},
    resume=True)
config = wandb.config

wandb.watch(model)
epoch = 0
step = 0
total_loss = 0

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [18]:
if True:
    copyfile([CHECKPOINT_PATH])
    checkpoint = torch.load(os.path.basename(CHECKPOINT_PATH))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    mean = checkpoint['mean']
    std = checkpoint['std']

In [17]:
model.load_state_dict(torch.load('/content/gdrive/Othercomputers/我的筆記型電腦/google drive/MAE Bs PJ/state_dict/epoch:1-3300:weight.pt'))

<All keys matched successfully>

# data

In [19]:
from torchvision import transforms
from utils import normalize_data, Node_Dataset, load_mean_std, get_time
from torch.utils.data import DataLoader, Dataset

In [20]:
# mean, std = (331.4973449707031, 130.28567504882812)

dataset = Node_Dataset(data_folder, mean, std)
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = Node_Dataset(test_folder, mean, std)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [21]:
print(mean, std)

tensor(331.1406) tensor(130.7411)


In [22]:
from utils import video_show, train_step, Evaluate, log

# train

In [ ]:
for epoch in range(Epochs):
    for i, batch in tqdm(enumerate(trainloader), total=len(trainloader)):

        loss = train_step(model,
                criterion,
                batch,
                optimizer)
        total_loss += loss

        if (i+1)%100 == 0:
            with torch.no_grad():
                evaluate_loss, visual_prediction, visual_target = Evaluate(model, criterion, testloader)

            log(CHECKPOINT_PATH, model, optimizer, step, mean, std)
            torch.save(model.state_dict(),os.path.join(model_savepath, f'epoch:{epoch+2}-{i+1}:weight.pt'))
            video = video_show(visual_prediction, visual_target, mean, std,'/content/output.mp4')
            wandb.log({"train_loss": total_loss,"test_loss": evaluate_loss, "visulize": wandb.Video(video, fps=5)}, step=step)
            step += 1
            total_loss = 0

    print(f'epoch: {epoch:>02}, time: {get_time()}')

  0%|          | 0/6406 [00:00<?, ?it/s]

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


epoch: 00, time: 14:59:6


  0%|          | 0/6406 [00:00<?, ?it/s]